In [2]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    import os
    import sys
    import numpy as np
    from elasticsearch import helpers
    print('All libraries loaded successfully')
    
except Exception as e:
    print(e)

All libraries loaded successfully


In [3]:
df = pd.read_csv('netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [5]:
print(df.shape)
df['show_id'].nunique()

(7787, 12)


7787

* show_id is a unique column

### Create elasticsearch instance

In [6]:
# connect to elastic search
web_address = 'http://localhost:9200/'
es = Elasticsearch(timeout = 600, hosts = web_address)

In [7]:
es.ping()

True

### Data cleaning

In [8]:
# check for number of null values in each column
df.isna().sum()

show_id            0
type               0
title              0
director        2389
cast             718
country          507
date_added        10
release_year       0
rating             7
duration           0
listed_in          0
description        0
dtype: int64

In [9]:
# drop all the null values
df = df.dropna()
df.isna().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [11]:
df.shape

(4808, 12)

### Convert data into APP format compatible with elasticsearch

In [12]:
df_dict = df.to_dict('records')

In [14]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [16]:
def generator(df):
    for item, line in enumerate(df):
        yield{
            '_index': 'first_elk',
            '_type': '_doc',
            '_id': line.get('show_id', None),
            '_source': {
                'title': line.get('title', ''),
                'director': line.get('director', ''),
                'cast': line.get('cast', ''),
                'country': line.get('country', ''),
            }
        }
    raise StopIteration

In [17]:
my_custom = generator(df_dict)

In [18]:
my_custom

<generator object generator at 0x000001E76F21DA98>

In [19]:
next(my_custom)

{'_index': 'first_elk',
 '_type': '_doc',
 '_id': 's2',
 '_source': {'title': '7:19',
  'director': 'Jorge Michel Grau',
  'cast': 'Demián Bichir, Héctor Bonilla, Oscar Serrano, Azalia Ortiz, Octavio Michel, Carmen Beato',
  'country': 'Mexico'}}